In [1]:
import gensim, logging

C:\Users\asus\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
from gensim import corpora, models, similarities

In [4]:
import os
import re
from collections import defaultdict

Сначала создаём корпус текстов.

In [5]:
#Так мы создаём список текстов, которые будут векторизоваться (простите за этот костыльный метод)
files = []
for i, file in enumerate(os.listdir('C://Users//asus//Desktop//Cybercorpora//Toys')):
    jj = os.path.abspath('Desktop\\Cybercorpora\\Toys\\' + file)
    dd = re.sub(r'\\', '//', jj)
    xx = open(jj, 'r', encoding = 'utf-8')
    ii = xx.read()
    files.append(ii)


In [7]:
#токенизируем. Потом можно сделать стоп-лист и вообще почистить тексты, но пока оставим такой игрушечный пример
texts = [[word for word in file.lower().split()] for file in files]

In [11]:
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [12]:
#векторизуем
dictionary = corpora.Dictionary(texts)
dictionary.save('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict')
print(dictionary)

2018-03-20 13:33:51,755 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-20 13:33:51,759 : INFO : built Dictionary(754 unique tokens: ['одной', 'проблемы', 'стал', 'тысяч', 'правовой']...) from 4 documents (total 3318 corpus positions)
2018-03-20 13:33:51,761 : INFO : saving Dictionary object under C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict, separately None
2018-03-20 13:33:51,769 : INFO : saved C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict


Dictionary(754 unique tokens: ['одной', 'проблемы', 'стал', 'тысяч', 'правовой']...)


In [133]:
corpus = [dictionary.doc2bow(text) for text in texts]
corpora.MmCorpus.serialize('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm', corpus)

2018-03-20 12:09:47,471 : INFO : storing corpus in Matrix Market format to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm
2018-03-20 12:09:47,473 : INFO : saving sparse matrix to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm
2018-03-20 12:09:47,474 : INFO : PROGRESS: saving document #0
2018-03-20 12:09:47,477 : INFO : saved 4x754 matrix, density=35.312% (1065/3016)
2018-03-20 12:09:47,487 : INFO : saving MmCorpus index to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm.index


In [149]:
#Переходим к сходству текстов
dictionary = corpora.Dictionary.load('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict')
corpus = corpora.MmCorpus('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm')
print(dictionary)

2018-03-20 12:18:54,266 : INFO : loading Dictionary object from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict
2018-03-20 12:18:54,269 : INFO : loaded C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.dict
2018-03-20 12:18:54,270 : INFO : loaded corpus index from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm.index
2018-03-20 12:18:54,271 : INFO : initializing cython corpus reader from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus.mm
2018-03-20 12:18:54,273 : INFO : accepted corpus with 4 documents, 754 features, 1065 non-zero entries


Dictionary(754 unique tokens: ['степени', 'тесно', 'внешней', 'состоит', 'religious']...)


In [135]:
lsi = models.LsiModel(corpus, id2word=dictionary, num_topics=2)

2018-03-20 12:09:54,953 : INFO : using serial LSI version on this node
2018-03-20 12:09:54,954 : INFO : updating model with new documents
2018-03-20 12:09:54,956 : INFO : preparing a new chunk of documents
2018-03-20 12:09:54,957 : INFO : using 100 extra samples and 2 power iterations
2018-03-20 12:09:54,959 : INFO : 1st phase: constructing (754, 102) action matrix
2018-03-20 12:09:54,960 : INFO : orthonormalizing (754, 102) action matrix
2018-03-20 12:09:54,975 : INFO : 2nd phase: running dense svd on (102, 4) matrix
2018-03-20 12:09:54,977 : INFO : computing the final decomposition
2018-03-20 12:09:54,978 : INFO : keeping 2 factors (discarding 4.119% of energy spectrum)
2018-03-20 12:09:54,980 : INFO : processed documents up to #4
2018-03-20 12:09:54,982 : INFO : topic #0(212.270): 0.623*"в" + 0.615*"и" + 0.193*"на" + 0.146*"с" + 0.102*"что" + 0.096*"по" + 0.091*"как" + 0.083*"не" + 0.076*"к" + 0.071*"-"
2018-03-20 12:09:54,984 : INFO : topic #1(65.727): 0.283*"в" + -0.214*"человека"

In [136]:
#тут я считаю сходство
doc = "Human computer interaction"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)


[(0, 0.016355223081294334), (1, -0.0509454109782947)]


In [137]:
#Никакого сходства, что разумно - другой язык
doc = "Ich hasse Sprachwissenschaft"
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[]


In [139]:
#Попробуем подать политологический текст
doc = open('C:/Users/asus/Desktop/Cybercorpora/textstocompare/abhaziya-na-perekrestke-geopoliticheskih-interesov.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 18.725760398105326), (1, 3.383763575027589)]


In [140]:
#А теперь из педагогики
doc = open('C:/Users/asus/Desktop/Cybercorpora/textstocompare/adaptivnost-i-kreativnost-v-stanovlenii-i-razvitii-tvorcheskoy-lichnosti-buduschego-uchitelya-v-sisteme-nepreryvnogo-pedagogicheskogo.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 119.34819809284976), (1, -9.592354497226939)]


Что-то он считает. Но что? 

In [142]:
doc = open('C:/Users/asus/Desktop/Cybercorpora/Toys/1-15-prava-cheloveka-v-traditsionnom-obschestve-vietnama.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 147.38810481559895), (1, -44.01744138755173)]


In [194]:
#Попробуем сравнить два одинаковых текста.
files2 = []
files2.append(open('C:/Users/asus/Desktop/Cybercorpora/Toys/5-8-determinanty-religioznogo-politicheskogo-ekstremizma.txt', 'r', encoding = 'utf-8').read())

In [195]:
texts2 = [[word2 for word2 in file2.lower().split()] for file2 in files2]
frequency2 = defaultdict(int)
for text2 in texts2:
    for token2 in text2:
        frequency2[token2] += 1

texts2 = [[token2 for token2 in text2 if frequency2[token2] > 1] for text2 in texts2]

In [196]:
dictionary2 = corpora.Dictionary(texts2)
dictionary2.save('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.dict')
print (dictionary2)

2018-03-20 13:10:31,413 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-20 13:10:31,415 : INFO : built Dictionary(186 unique tokens: ['норм', 'рассматривается', 'religious', 'устойчивый,', 'по']...) from 1 documents (total 662 corpus positions)
2018-03-20 13:10:31,416 : INFO : saving Dictionary object under C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.dict, separately None
2018-03-20 13:10:31,420 : INFO : saved C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.dict


Dictionary(186 unique tokens: ['норм', 'рассматривается', 'religious', 'устойчивый,', 'по']...)


In [197]:
corpus2 = [dictionary2.doc2bow(text2) for text2 in texts2]
corpora.MmCorpus.serialize('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm', corpus2)
print(corpus2)

2018-03-20 13:10:33,394 : INFO : storing corpus in Matrix Market format to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm
2018-03-20 13:10:33,395 : INFO : saving sparse matrix to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm
2018-03-20 13:10:33,396 : INFO : PROGRESS: saving document #0
2018-03-20 13:10:33,398 : INFO : saved 1x186 matrix, density=100.000% (186/186)
2018-03-20 13:10:33,401 : INFO : saving MmCorpus index to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm.index


[[(0, 2), (1, 2), (2, 6), (3, 2), (4, 2), (5, 2), (6, 4), (7, 2), (8, 2), (9, 3), (10, 11), (11, 4), (12, 4), (13, 7), (14, 2), (15, 9), (16, 2), (17, 2), (18, 4), (19, 3), (20, 2), (21, 2), (22, 2), (23, 2), (24, 2), (25, 45), (26, 2), (27, 2), (28, 2), (29, 2), (30, 2), (31, 2), (32, 2), (33, 6), (34, 2), (35, 4), (36, 2), (37, 2), (38, 2), (39, 2), (40, 2), (41, 2), (42, 5), (43, 3), (44, 2), (45, 4), (46, 2), (47, 2), (48, 5), (49, 2), (50, 4), (51, 2), (52, 2), (53, 2), (54, 51), (55, 2), (56, 2), (57, 2), (58, 2), (59, 2), (60, 7), (61, 3), (62, 8), (63, 2), (64, 2), (65, 2), (66, 2), (67, 2), (68, 2), (69, 3), (70, 2), (71, 2), (72, 3), (73, 3), (74, 2), (75, 2), (76, 10), (77, 2), (78, 2), (79, 2), (80, 2), (81, 4), (82, 2), (83, 2), (84, 3), (85, 3), (86, 2), (87, 2), (88, 2), (89, 2), (90, 2), (91, 3), (92, 6), (93, 3), (94, 2), (95, 2), (96, 2), (97, 3), (98, 2), (99, 2), (100, 16), (101, 3), (102, 2), (103, 3), (104, 2), (105, 2), (106, 6), (107, 12), (108, 2), (109, 2), (1

In [198]:
dictionary2 = corpora.Dictionary.load('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.dict')
corpus2 = corpora.MmCorpus('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm')
print(corpus2)

2018-03-20 13:10:35,668 : INFO : loading Dictionary object from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.dict
2018-03-20 13:10:35,670 : INFO : loaded C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.dict
2018-03-20 13:10:35,671 : INFO : loaded corpus index from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm.index
2018-03-20 13:10:35,673 : INFO : initializing cython corpus reader from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus2.mm
2018-03-20 13:10:35,674 : INFO : accepted corpus with 1 documents, 186 features, 186 non-zero entries


MmCorpus(1 documents, 186 features, 186 non-zero entries)


In [199]:
lsi = models.LsiModel(corpus2, id2word=dictionary2, num_topics=2)

2018-03-20 13:10:37,636 : INFO : using serial LSI version on this node
2018-03-20 13:10:37,637 : INFO : updating model with new documents
2018-03-20 13:10:37,639 : INFO : preparing a new chunk of documents
2018-03-20 13:10:37,640 : INFO : using 100 extra samples and 2 power iterations
2018-03-20 13:10:37,642 : INFO : 1st phase: constructing (186, 102) action matrix
2018-03-20 13:10:37,643 : INFO : orthonormalizing (186, 102) action matrix
2018-03-20 13:10:37,650 : INFO : 2nd phase: running dense svd on (102, 1) matrix
2018-03-20 13:10:37,651 : INFO : computing the final decomposition
2018-03-20 13:10:37,653 : INFO : keeping 1 factors (discarding 0.000% of energy spectrum)
2018-03-20 13:10:37,654 : INFO : processed documents up to #1
2018-03-20 13:10:37,655 : INFO : topic #0(86.475): 0.590*"и" + 0.520*"в" + 0.231*"с" + 0.185*"по" + 0.139*"политического" + 0.127*"of" + 0.127*"религиозного" + 0.116*"на" + 0.116*"религиозный" + 0.104*"экстремизм"


In [200]:
doc = open('C:/Users/asus/Desktop/Cybercorpora/Toys/5-8-determinanty-religioznogo-politicheskogo-ekstremizma.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary2.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 86.47543003651363)]


In [201]:
doc = open('C:/Users/asus/Desktop/Cybercorpora/textstocompare/5-8-determinanty-religioznogo-politicheskogo-ekstremizma.txt', 'r', encoding = 'utf-8').read()
vec_bow = dictionary2.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 73.07277913890472)]


In [202]:
files3 = []
files3.append(open('C:/Users/asus/Desktop/Cybercorpora/textstocompare/bazovyye-kontsepty.txt', 'r', encoding = 'cp1251').read())
texts3 = [[word3 for word3 in file3.lower().split()] for file3 in files3]
frequency3 = defaultdict(int)
for text3 in texts3:
    for token3 in text3:
        frequency3[token3] += 1

texts3 = [[token3 for token3 in text3 if frequency3[token3] > 1] for text3 in texts3]
dictionary3 = corpora.Dictionary(texts3)
dictionary3.save('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.dict')

corpus3 = [dictionary3.doc2bow(text3) for text3 in texts3]
corpora.MmCorpus.serialize('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.mm', corpus3)

2018-03-20 13:11:24,547 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2018-03-20 13:11:24,549 : INFO : built Dictionary(231 unique tokens: ['фразеологический', 'процессе', 'вслед', 'человека', 'функционирования']...) from 1 documents (total 886 corpus positions)
2018-03-20 13:11:24,550 : INFO : saving Dictionary object under C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.dict, separately None
2018-03-20 13:11:24,553 : INFO : saved C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.dict
2018-03-20 13:11:24,555 : INFO : storing corpus in Matrix Market format to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.mm
2018-03-20 13:11:24,556 : INFO : saving sparse matrix to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.mm
2018-03-20 13:11:24,557 : INFO : PROGRESS: saving document #0
2018-03-20 13:11:24,559 : INFO : saved 1x231 matrix, density=100.000% (231/231)
2018-03-20 13:11:24,562 : INFO : saving MmCorpus index to C:/Users/asus/Desktop/Cybercorpora/Vectors/corpu

In [203]:
dictionary3 = corpora.Dictionary.load('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.dict')
corpus3 = corpora.MmCorpus('C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.mm')

2018-03-20 13:11:38,417 : INFO : loading Dictionary object from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.dict
2018-03-20 13:11:38,419 : INFO : loaded C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.dict
2018-03-20 13:11:38,420 : INFO : loaded corpus index from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.mm.index
2018-03-20 13:11:38,421 : INFO : initializing cython corpus reader from C:/Users/asus/Desktop/Cybercorpora/Vectors/corpus3.mm
2018-03-20 13:11:38,422 : INFO : accepted corpus with 1 documents, 231 features, 231 non-zero entries


In [204]:
lsi = models.LsiModel(corpus3, id2word=dictionary3, num_topics=2)

2018-03-20 13:11:41,084 : INFO : using serial LSI version on this node
2018-03-20 13:11:41,085 : INFO : updating model with new documents
2018-03-20 13:11:41,087 : INFO : preparing a new chunk of documents
2018-03-20 13:11:41,088 : INFO : using 100 extra samples and 2 power iterations
2018-03-20 13:11:41,089 : INFO : 1st phase: constructing (231, 102) action matrix
2018-03-20 13:11:41,090 : INFO : orthonormalizing (231, 102) action matrix
2018-03-20 13:11:41,096 : INFO : 2nd phase: running dense svd on (102, 1) matrix
2018-03-20 13:11:41,097 : INFO : computing the final decomposition
2018-03-20 13:11:41,098 : INFO : keeping 1 factors (discarding 0.000% of energy spectrum)
2018-03-20 13:11:41,099 : INFO : processed documents up to #1
2018-03-20 13:11:41,101 : INFO : topic #0(121.515): 0.609*"и" + 0.593*"в" + 0.189*"с" + 0.181*"как" + 0.115*"его" + 0.107*"или" + 0.099*"для" + 0.099*"к" + 0.091*"-" + 0.074*"на"


In [205]:
doc = open('C:/Users/asus/Desktop/Cybercorpora/textstocompare/tsennostnaya-kontseptosphera.txt', 'r', encoding = 'cp1251').read()
vec_bow = dictionary3.doc2bow(doc.lower().split())
vec_lsi = lsi[vec_bow]
print(vec_lsi)

[(0, 107.90399111677931)]
